# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import datetime

In [2]:
import api_token  # handle iexcloud API tokens

env = "cloud" # "sandbox"  # 
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(env=env), api_token.get_api_token(env=env)

In [3]:
WRITE_ALL_DF = False

TOP_N = 150
portfolio_size = 200000
filename = "iexcloud"

In [ ]:
today = datetime.datetime.now()
xlsx_file = f"spreadsheet_{filename}_top{TOP_N}_{today.strftime('%Y-%m-%d')}.xlsx"
xlsx_file2 = f"spreadsheet_{filename}_all_{today.strftime('%Y-%m-%d')}.xlsx"
watchlist_file = f"watchlist_{filename}_top{TOP_N}_{today.strftime('%Y-%m-%d')}.csv"

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [5]:
api_url = f'{BASE_URL}/stable/ref-data/iex/symbols?token={IEX_CLOUD_API_TOKEN}'
api_url
data = requests.get(api_url).json()

In [6]:
stocks = [d['symbol']  for d in data if d['isEnabled'] 
           and "." not in d['symbol'] 
           and "-" not in d['symbol'] 
           and "+" not in d['symbol']
           and "=" not in d['symbol']
          ]

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 5-day price returns
* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [9]:
hqm_columns = [
                'Ticker', 
                'Name',
                'Number of Shares to Buy', 
                'Price', 
                'Five-Day Price Return',
                'One-Month Price Return',
                'Three-Month Price Return',
                'Six-Month Price Return',
                'One-Year Price Return', 
                'Five-Day Return Percentile',
                'One-Month Return Percentile',
                'Three-Month Return Percentile',
                'Six-Month Return Percentile',
                'One-Year Return Percentile',
                'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                            pd.Series([symbol, 
                                       data[symbol]['stats']['companyName'],
                                       'N/A',
                                       data[symbol]['quote']['latestPrice'],
                                       data[symbol]['stats']['day5ChangePercent'],
                                       data[symbol]['stats']['month1ChangePercent'],
                                       data[symbol]['stats']['month3ChangePercent'],
                                       data[symbol]['stats']['month6ChangePercent'],
                                       data[symbol]['stats']['year1ChangePercent'],
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A'
                                       ], 
                                      index = hqm_columns), 
                            ignore_index = True)
        except Exception as ex:
            print(f"symbol = {symbol}\n Error={str(ex)}")
        
hqm_dataframe.columns

symbol = AFINO
 Error='quote'
symbol = CBO
 Error='CBO'
symbol = CBX
 Error='CBX'
symbol = CVLB
 Error='quote'
symbol = MDWT
 Error='quote'
symbol = NTEST
 Error='NTEST'
symbol = POWW
 Error='quote'
symbol = ZAZZT
 Error='ZAZZT'
symbol = ZBZX
 Error='ZBZX'
symbol = ZBZZT
 Error='ZBZZT'
symbol = ZCZZT
 Error='ZCZZT'
symbol = ZEXIT
 Error='ZEXIT'
symbol = ZIEXT
 Error='ZIEXT'
symbol = ZJZZT
 Error='ZJZZT'
symbol = ZTEST
 Error='ZTEST'
symbol = ZVV
 Error='ZVV'
symbol = ZVVV
 Error='ZVVV'
symbol = ZWZZT
 Error='ZWZZT'
symbol = ZXIET
 Error='ZXIET'
symbol = ZXZZT
 Error='ZXZZT'
symbol = BTCUSDT
 Error='NoneType' object is not subscriptable
symbol = EOSUSDT
 Error='NoneType' object is not subscriptable
symbol = ETHUSDT
 Error='NoneType' object is not subscriptable
symbol = BNBUSDT
 Error='NoneType' object is not subscriptable
symbol = ONTUSDT
 Error='NoneType' object is not subscriptable
symbol = BCCUSDT
 Error='NoneType' object is not subscriptable
symbol = ADAUSDT
 Error='NoneType' object

Index(['Ticker', 'Name', 'Number of Shares to Buy', 'Price',
       'Five-Day Price Return', 'Five-Day Return Percentile',
       'One-Month Price Return', 'One-Month Return Percentile',
       'Three-Month Price Return', 'Three-Month Return Percentile',
       'Six-Month Price Return', 'Six-Month Return Percentile',
       'One-Year Price Return', 'One-Year Return Percentile', 'HQM Score'],
      dtype='object')

In [10]:
hqm_dataframe

,Ticker,Name,Number of Shares to Buy,Price,Five-Day Price Return,Five-Day Return Percentile,One-Month Price Return,One-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,N/A,117.37,0.00890869,N/A,0.0621336,N/A,0.202408,N/A,0.332584,N/A,0.394429,N/A,N/A
1,AA,Alcoa Corp,N/A,21.61,0.0160846,N/A,0.174814,N/A,0.797561,N/A,0.925958,N/A,0.0414508,N/A,N/A
2,AAA,Listed Funds Trust,N/A,25.08,0.000399042,N/A,0.0020785,N/A,0.00591431,N/A,0.00230687,N/A,0.00230687,N/A,N/A
3,AAAU,Goldman Sachs Physical Gold ETF Shares,N/A,18.53,0.0258242,N/A,0.00107239,N/A,-0.0199475,N/A,0.0729885,N/A,0.266536,N/A,N/A
4,AACG,ATA Creativity Global,N/A,1.2,0.025641,N/A,0.0434783,N/A,0.132075,N/A,0.263158,N/A,-0.20398,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8638,ZVO,Zovio Inc,N/A,4.11,-0.0423529,N/A,0.00992556,N/A,-0.0578704,N/A,0.541667,N/A,1.05556,N/A,N/A
8639,ZVZZT,,N/A,10.02,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
8640,ZYME,Zymeworks Inc,N/A,53.05,0.0205745,N/A,0.114928,N/A,0.198814,N/A,0.401386,N/A,0.161732,N/A,N/A
8641,ZYNE,Zynerba Pharmaceuticals Inc,N/A,3.415,-0.146402,N/A,-0.184834,N/A,-0.0171429,N/A,-0.430464,N/A,-0.446055,N/A,N/A


In [13]:
time_periods = [
                'Five-Day',
                'One-Month',
                'Three-Month',
                'Six-Month',
                'One-Year'
                ]
price_return_cols = [f'{time_period} Price Return' for time_period in time_periods]

price_return_cols

['Five-Day Price Return',
 'One-Month Price Return',
 'Three-Month Price Return',
 'Six-Month Price Return',
 'One-Year Price Return']

In [14]:
hqm_dataframe = hqm_dataframe.dropna(how='any', 
                                     subset=price_return_cols)

In [12]:
hqm_dataframe[hqm_dataframe["One-Year Price Return"].isnull()]

,Ticker,Name,Number of Shares to Buy,Price,Five-Day Price Return,Five-Day Return Percentile,One-Month Price Return,One-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,HQM Score
3373,GOEV,Canoo Inc,N/A,19.01,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
3374,GOEVW,Canoo Inc,N/A,5.17,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
5251,MVNR,Mavenir Private Holdings II Ltd.,N/A,17.5,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
5753,OPINL,Office Properties Income Trust,N/A,26.28,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
6156,PPBT,Purple Biotech Ltd,N/A,4.17,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
6207,PRS,"Prudential Financial, Inc.",N/A,28.39,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
6462,RCB,Ready Capital Corp.,N/A,24.144,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
7999,USWSW,"US Well Services, Inc.",N/A,0.0288,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
8489,XL,XL Fleet Corporation,N/A,17.52,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
8639,ZVZZT,,N/A,10.02,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`
* `Five-Day Price Return`

Here's how we'll do this:

In [15]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

/home/wengong/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-15-b53cad76fb7b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100


,Ticker,Name,Number of Shares to Buy,Price,Five-Day Price Return,Five-Day Return Percentile,One-Month Price Return,One-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,N/A,117.37,0.00890869,0.556238,0.0621336,0.603382,0.202408,0.599907,0.332584,0.726051,0.394429,0.81258,N/A
1,AA,Alcoa Corp,N/A,21.61,0.0160846,0.642534,0.174814,0.845824,0.797561,0.945094,0.925958,0.930615,0.0414508,0.496583,N/A
2,AAA,Listed Funds Trust,N/A,25.08,0.000399042,0.40183,0.0020785,0.238735,0.00591431,0.170509,0.00230687,0.18522,0.00230687,0.409128,N/A
3,AAAU,Goldman Sachs Physical Gold ETF Shares,N/A,18.53,0.0258242,0.722576,0.00107239,0.229932,-0.0199475,0.102977,0.0729885,0.320978,0.266536,0.758137,N/A
4,AACG,ATA Creativity Global,N/A,1.2,0.025641,0.720781,0.0434783,0.5128,0.132075,0.459516,0.263158,0.648037,-0.20398,0.16796,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8637,ZUO,Zuora Inc,N/A,14,0.0771757,0.89239,0.134948,0.797869,0.28,0.706186,0.00845503,0.195876,-0.0876217,0.2692,N/A
8638,ZVO,Zovio Inc,N/A,4.11,-0.0423529,0.095216,0.00992556,0.289355,-0.0578704,0.0783042,0.541667,0.851848,1.05556,0.921696,N/A
8640,ZYME,Zymeworks Inc,N/A,53.05,0.0205745,0.682613,0.114928,0.764161,0.198814,0.595158,0.401386,0.780146,0.161732,0.684003,N/A
8641,ZYNE,Zynerba Pharmaceuticals Inc,N/A,3.415,-0.146402,0.0105409,-0.184834,0.013321,-0.0171429,0.106452,-0.430464,0.0210819,-0.446055,0.0589598,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [16]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

/home/wengong/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-16-f94fc74bc0fa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)


# Selecting the Best Momentum Stocks

As before, we can identify the 100 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 100 entries.

In [17]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)

if WRITE_ALL_DF:
    hqm_dataframe_all = hqm_dataframe.copy()

hqm_dataframe = hqm_dataframe[:(TOP_N+1)]

<ipython-input-17-d8b1b5c16398>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [18]:
# portfolio_input()

In [19]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

In [20]:
position_size

1324.5033112582782

In [21]:
hqm_dataframe.reset_index(drop = True, inplace = True)

In [22]:
len(hqm_dataframe['Ticker'])

151

In [24]:
for i in range(len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])

/home/wengong/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-24-210e8b19d47d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])


In [23]:
hqm_dataframe

,Ticker,Name,Number of Shares to Buy,Price,Five-Day Price Return,Five-Day Return Percentile,One-Month Price Return,One-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,HQM Score
0,BRPAW,Big Rock partners Acquisition Corp,N/A,6.55,1.21925,0.999421,12.8333,0.999884,11.9688,0.999305,9.98174,0.997799,22.0556,0.997452,0.998772
1,FUBO,fuboTV Inc,N/A,62,0.822468,0.997799,1.09218,0.990617,114.07,0.999768,114.07,0.999768,114.07,0.999421,0.997475
2,MARA,Marathon Patent Group Inc,N/A,13.95,1.07581,0.998726,2.39233,0.998031,5.08466,0.997104,12.6628,0.998147,11.028,0.995019,0.997405
3,BEEMW,Beam Global,N/A,44.04,0.614961,0.995714,1.32893,0.993513,5.62256,0.997567,18.8378,0.998842,52.7073,0.998957,0.996919
4,QS,QuantumScape Corp,N/A,131.67,0.691622,0.996757,3.71202,0.999189,4.17166,0.996177,8.58586,0.996988,8.58586,0.992702,0.996363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,U,Unity Software Inc,N/A,172.29,0.127113,0.94243,0.346661,0.933974,1.26658,0.976949,1.41946,0.963744,1.41946,0.941272,0.951674
147,CYTHW,Cyclo Therapeutics Inc,N/A,1.57,1.17647,0.999305,0.85,0.984999,0.85,0.950075,0.85,0.919611,0.85,0.903162,0.951431
148,STON,StoneMor Inc,N/A,2.45,0.379747,0.990154,0.847458,0.984826,1.18196,0.973011,1.44944,0.96467,0.503448,0.843739,0.95128
149,TAN,Invesco Capital Management LLC,N/A,103.13,0.158482,0.958647,0.277568,0.911618,0.806047,0.946484,1.83492,0.974053,2.25853,0.965365,0.951234


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Name', string_template], 
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price', dollar_template],
                    'E': ['Five-Day Price Return', percent_template], 
                    'F': ['One-Month Price Return', percent_template], 
                    'G': ['Three-Month Price Return', percent_template],
                    'H': ['Six-Month Price Return', percent_template],
                    'I': ['One-Year Price Return', percent_template],
                    'J': ['Five-Day Return Percentile', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['Three-Month Return Percentile', percent_template],
                    'M': ['Six-Month Return Percentile', percent_template],
                    'N': ['One-Year Return Percentile', percent_template], 
                    'O': ['HQM Score', percent_template]
                    }

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [27]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

writer.save()

In [ ]:
if WRITE_ALL_DF:
    writer = pd.ExcelWriter(xlsx_file2, engine='xlsxwriter')
    hqm_dataframe_all.to_excel(writer, sheet_name='Momentum Strategy', index = False)

    for column in column_formats.keys():
        writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
        writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

    writer.save()

### save watchlist

In [31]:
tickers = hqm_dataframe[['Ticker']]
tickers.to_csv(watchlist_file, index=False)